In [1]:
from keras import models
from keras import layers
from keras import optimizers, losses, metrics

import numpy as np

Using TensorFlow backend.


<br>
<br>

# 데이터 생성

In [2]:
# 캐릭터 글자 목록
# S -> Start, E -> End, P -> Padding
char_list = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz사랑얼굴주택희망나무바위']

# 캐릭터 사전 생성
char_to_idx = {c: i for i, c in enumerate(char_list)}
dic_len = len(char_to_idx)

print(char_to_idx)


{'S': 0, 'E': 1, 'P': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28, '사': 29, '랑': 30, '얼': 31, '굴': 32, '주': 33, '택': 34, '희': 35, '망': 36, '나': 37, '무': 38, '바': 39, '위': 40}


In [3]:
# 단어 목록
# 앞의 영어를 뒤의 한글로 번역
word_list = [['love', '사랑'], ['face', '얼굴'],
            ['home', '주택'], ['hope', '희망'],
            ['tree', '나무'], ['rock', '바위']]

In [4]:
# 배치 데이터 생성
def make_batch(word_list):
    
    encoder_input = []
    decoder_input = []
    decoder_target = []

    for words in word_list:
        # 인코더 입력 단어를 인덱스로 변환
        # Ex) l,o,v,e
        enc_input = [char_to_idx[c] for c in words[0]]

        # 디코더 입력 단어를 인덱스로 변환
        # 제일 앞에 시작 태그 삽입
        # Ex) S,사,랑
        dec_input = [char_to_idx[c] for c in ('S' + words[1])]
        
        # 디코더 목표 캐릭터를 인덱스로 변환
        # 제일 끝에 종료 태그 삽입
        # Ex) 사,랑,E
        dec_target = [char_to_idx[c] for c in (words[1] + 'E')]

        # 원핫인코딩으로 변환
        encoder_input.append(np.eye(dic_len)[enc_input])
        decoder_input.append(np.eye(dic_len)[dec_input])
        decoder_target.append(np.eye(dic_len)[dec_target])

    return np.array(encoder_input), np.array(decoder_input), np.array(decoder_target)

In [5]:
# 배치 생성
x_encoder, x_decoder, y_decoder = make_batch(word_list)

In [6]:
x_encoder[0]


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [7]:
x_decoder[0]


array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [8]:
y_decoder[0]


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.]])

<br>
<br>

# 모델 생성

In [9]:
#--------------------------------------------
# 인코더 정의
#--------------------------------------------

# 입력 문장의 인덱스 시퀀스를 입력으로 받음
encoder_inputs = layers.Input(shape=(None,dic_len))

# return_state가 True면 상태값 리턴
# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
encoder_outputs, state_h, state_c = layers.LSTM(64,
                                                dropout=0.1,
                                                recurrent_dropout=0.5,
                                                return_state=True)(encoder_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
encoder_states = [state_h, state_c]



#--------------------------------------------
# 디코더 정의
#--------------------------------------------

# 목표 문장의 인덱스 시퀀스를 입력으로 받음
decoder_inputs = layers.Input(shape=(None,dic_len))

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임 스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리하기 위함
decoder_lstm = layers.LSTM(64,
                           dropout=0.1,
                           recurrent_dropout=0.5,
                           return_state=True,
                           return_sequences=True)

# initial_state를 인코더의 상태로 초기화
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = layers.Dense(dic_len, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)



#--------------------------------------------
# 모델 정의
#--------------------------------------------

# 입력과 출력으로 함수형 API 모델 생성
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])    

<br>
<br>

# 훈련 및 테스트

In [10]:
# 훈련 시작
model.fit([x_encoder, x_decoder],
           y_decoder,
           epochs=200,
           batch_size=1)


Epoch 1/200
6/6 [==============================] - 2s 278ms/step - loss: 3.7125 - acc: 0.0556
Epoch 2/200
6/6 [==============================] - 0s 15ms/step - loss: 3.6590 - acc: 0.2222
Epoch 3/200
6/6 [==============================] - 0s 14ms/step - loss: 3.6001 - acc: 0.3333
Epoch 4/200
6/6 [==============================] - 0s 15ms/step - loss: 3.5365 - acc: 0.3333
Epoch 5/200
6/6 [==============================] - 0s 14ms/step - loss: 3.4779 - acc: 0.3333
Epoch 6/200
6/6 [==============================] - 0s 14ms/step - loss: 3.3799 - acc: 0.3333
Epoch 7/200
6/6 [==============================] - 0s 18ms/step - loss: 3.2201 - acc: 0.3333
Epoch 8/200
6/6 [==============================] - 0s 15ms/step - loss: 3.0477 - acc: 0.3333
Epoch 9/200
6/6 [==============================] - 0s 18ms/step - loss: 2.8509 - acc: 0.3333
Epoch 10/200
6/6 [==============================] - 0s 16ms/step - loss: 2.5012 - acc: 0.3333
Epoch 11/200
6/6 [==============================] - 0s 14ms/step - l

6/6 [==============================] - 0s 14ms/step - loss: 0.7811 - acc: 0.8889
Epoch 88/200
6/6 [==============================] - 0s 14ms/step - loss: 0.6904 - acc: 0.8889
Epoch 89/200
6/6 [==============================] - 0s 15ms/step - loss: 0.8954 - acc: 0.8889
Epoch 90/200
6/6 [==============================] - 0s 14ms/step - loss: 0.8249 - acc: 0.8333
Epoch 91/200
6/6 [==============================] - 0s 15ms/step - loss: 0.7230 - acc: 0.9444
Epoch 92/200
6/6 [==============================] - 0s 14ms/step - loss: 0.6971 - acc: 0.8889
Epoch 93/200
6/6 [==============================] - 0s 13ms/step - loss: 0.7486 - acc: 0.8333
Epoch 94/200
6/6 [==============================] - 0s 14ms/step - loss: 0.6003 - acc: 1.0000
Epoch 95/200
6/6 [==============================] - 0s 13ms/step - loss: 0.6673 - acc: 0.9444
Epoch 96/200
6/6 [==============================] - 0s 14ms/step - loss: 0.6720 - acc: 0.8889
Epoch 97/200
6/6 [==============================] - 0s 15ms/step - loss: 

6/6 [==============================] - 0s 12ms/step - loss: 0.2231 - acc: 0.9444
Epoch 174/200
6/6 [==============================] - 0s 14ms/step - loss: 0.2439 - acc: 0.9444
Epoch 175/200
6/6 [==============================] - 0s 15ms/step - loss: 0.1800 - acc: 1.0000
Epoch 176/200
6/6 [==============================] - 0s 12ms/step - loss: 0.1747 - acc: 1.0000
Epoch 177/200
6/6 [==============================] - 0s 13ms/step - loss: 0.1756 - acc: 1.0000
Epoch 178/200
6/6 [==============================] - 0s 13ms/step - loss: 0.1600 - acc: 1.0000
Epoch 179/200
6/6 [==============================] - 0s 16ms/step - loss: 0.1560 - acc: 1.0000
Epoch 180/200
6/6 [==============================] - 0s 11ms/step - loss: 0.1719 - acc: 0.9444
Epoch 181/200
6/6 [==============================] - 0s 13ms/step - loss: 0.2065 - acc: 1.0000
Epoch 182/200
6/6 [==============================] - 0s 16ms/step - loss: 0.1966 - acc: 0.9444
Epoch 183/200
6/6 [==============================] - 0s 15ms/ste

In [11]:
# 번역 수행
def translate(word):
    
    # 영어/한글 배열 생성
    # 한글은 정답을 모르기 때문에 패딩으로 채움
    words = [word, 'P' * len(word)]

    # 배치 데이터 생성
    x_encoder, x_decoder, y_decoder = make_batch([words])
    
    # 예측 수행
    # 원핫인코딩으로 결과 나옴
    results = model.predict([x_encoder, x_decoder])

    # 2축을 기준으로 최대값의 인덱스 구함
    results = np.argmax(results, 2) 

    # 인덱스를 캐릭터로 변환
    decoded = [char_list[i] for i in results[0]]

    # 종료 태그인 'E' 이후의 글자들을 제거하고 문자열 생성
    end = decoded.index('E')
    translated = ''.join(decoded[:end])

    return translated

In [12]:
print('love ->', translate('love'))
print('lovi ->', translate('lovi'))
print('litr ->', translate('litr'))
print('hope ->', translate('hope'))
print('hopu ->', translate('hopu'))
print('hufe ->', translate('hufe'))


love -> 사랑
lovi -> 사랑
litr -> 나
hope -> 희망
hopu -> 희망
hufe -> 희
